In [1]:
# -*- coding: utf-8 -*-
import sys
sys.path.append("/home/ubuntu/ai_community/chat_sup")
import config

import sqlite3
import pandas as pd
import numpy as np
import shelve

In [2]:
class SQLighter:
    def __init__(self, database):
        self.connection = sqlite3.connect(database)
        self.cursor = self.connection.cursor()
        
        # поля для основной таблички events
        self.fields = [['date', 'datetime'],['name', 'text'], ['descr', 'text'], ['max_memb_count', 'integer']]
        # поля для каждой из табличек
        self.fields_event = [['user_id', 'text'], ['status','text']]
    def create_table(self, name, fiels):
        self.cursor.execute('create table {0} ({1})'.format(name, ', '.join([' '.join(t) for t in fiels])))
        
    def insert_into_table(self, name_of_table, columns, values):
        self.cursor.execute('INSERT INTO {0} {1} VALUES {2}'.format(name_of_table,
                                                                      columns,
                                                                      values))
        self.connection.commit()
    def create_event(self, event_object):
        # Для каждого нового события создается новая табличка что бы записывать участников
        self.create_table(event_object['name'], [['user_id', 'text'], ['status','text']])
        # Для каждого события в табличку events добавляется строка
        self.insert_into_table('events', 
                               columns = tuple(pd.Series(event_object).dropna().index), 
                               values = tuple(pd.Series(event_object).dropna().values))
    def add_user(self, user_id): # метод, который добавляет пользователя в табличку users
        self.insert_into_table(name_of_table = 'users', columns="('user_id')", values=user_id)
        
    def check_user(self, user_id): # метод, который проверяет, есть ли пользователь в табличке users
        try:
            t = pd.read_sql('select count(*) from users u where u.user_id = {}'.format(user_id), self.connection)
            if t.values[0][0] == 1:
                return True
            elif t.values[0][0] >1:
                return 'WTF?'
            else:
                return False
        except:
            return False
    def check_event(self, event): # метод для проверки того, что событие есть вообще существует
        try:
            t = pd.read_sql("select count(*) from events e where e.name = '{}'".format(event), self.connection)
            if t.values[0][0] == 1:
                return True
            elif t.values[0][0] == 1:
                return 'WTF?'
            else:
                return False
        except:
            return False
    def reg_user_for_event(self, user, event):
        self.insert_into_table(event, columns=('user_id', 'status'), values=(user, 'first_reg'))
    def check_moderator(self, user_id):
        return(pd.read_sql('select is_moderator from {0} where user_id = {1}'.format('users', user_id), 
                           self.connection).values[0][0])

In [3]:
db = SQLighter('../chat_sup/aievents.db')

In [4]:
pd.read_sql('select * from {0} '.format('events', ), db.connection)

,date,name,descr,max_memb_count
0,2009-05-03,new_event_1,Not so interesting but u should go,20
1,2009-03-15,new_event_2,very interesting,9
2,2018-05-15,new_event_3,Big data so big,9
3,2018-10-01,new_event_4,do u have some... data?,34
4,2018-09-01,first_september,"все дети идут в школу, почему бы тебе тоже не...",100


In [9]:
pd.read_sql('select * from {0} '.format('users', ), db.connection)

,user_id,birth_date,sex,is_moderator
0,354460030,None,None,True
1,959120,None,None,True


In [5]:
pd.read_sql('select is_moderator from {0} where user_id = {1}'.format('users', 354460030), db.connection).values[0][0]

In [6]:
pd.read_sql('select * from {0} order by value_date'.format('anketa'), db.connection)

,user_id,anketa_field,value,value_date
0,354460030,first_name,Yury,2018-05-05T20:46:42.309576+03:00
1,354460030,last_name,Simonov,2018-05-05T20:46:42.312470+03:00
2,354460030,username,d0naId,2018-05-05T20:46:42.315460+03:00
3,354460030,name,Юра,2018-05-05T21:07:31.176764+03:00
4,354460030,surname,Симонов,2018-05-05T21:07:36.166273+03:00
5,354460030,edu,МГУ,2018-05-05T21:07:39.838114+03:00
6,354460030,work,Альфабанк,2018-05-05T21:07:45.516780+03:00
7,354460030,name,Юра,2018-05-05T22:57:02.011683+03:00
8,354460030,surname,Симонов,2018-05-05T22:57:07.322806+03:00
9,354460030,edu,МГУ,2018-05-05T22:57:10.423975+03:00


In [8]:
db.cursor.execute("UPDATE users SET is_moderator = 'True' WHERE user_id=354460030")
db.connection.commit()

In [14]:
db.cursor.execute("DELETE from events")
db.connection.commit()

In [19]:
# этот словарь заполняет менеджер
event_object_1 = {
    'date':'2009-05-03', 'name':'new_event_1', 'descr':'Not so interesting but u should go', 'max_memb_count':20
}
# тест создания нового события
db.create_event(event_object_1)

# этот словарь заполняет менеджер
event_object_2 = {
    'date':'2009-03-15', 'name':'new_event_2', 'descr':'very interesting', 'max_memb_count':9
}
# тест создания нового события
db.create_event(event_object_2)

# этот словарь заполняет менеджер
event_object_3 = {
    'date':'2018-05-15', 'name':'new_event_3', 'descr':'Big data so big', 'max_memb_count':9
}
# тест создания нового события
db.create_event(event_object_3)

# этот словарь заполняет менеджер
event_object_4 = {
    'date':'2018-10-01', 'name':'new_event_4', 'descr':'do u have some... data?', 'max_memb_count':34
}
# тест создания нового события
db.create_event(event_object_4)

In [8]:
# тест создания нового пользователя
db.add_user('(\''+'1234'+'\')')

In [4]:
# создаю основную табличку events
db.create_table('events', db.fields)

# создаю основную табличку users
db.create_table('users', [['user_id', 'text'],['birth_date', 'datetime'],['sex','BLOB'],['is_moderator','BLOB']])

# создаю основную табличку anketa
fields_anketa = [['user_id','text'], ['anketa_field','text'], ['value', 'text'], ['value_date', 'text']]
db.create_table('anketa', fields_anketa)

In [7]:
quetions_n = {'name':1,
'surname':2,
'edu':3,
'work':4,
'position':5}

quetions = {'name':'Отлично! Назови, пожалуйста, свое имя...',
'surname':'Я в экстазе! Теперь напиши, пожалуйста, свою фамилию...',
'edu':'Раз уж мы так разоткровенничили, то где ты учился?',
'work':'Молодец! А где работаешь?',
'position':'Какая у тебя должность?'}

In [9]:
shelve.open(config.action_shelve)[str(354460030)] in quetions_n.keys()

True

In [11]:
quetions_n.keys()

dict_keys(['name', 'surname', 'edu', 'work', 'position'])

In [8]:
quetions_n = {'name':1,
'surname':2,
'edu':3,
'work':4,
'position':5}

quetions = {'name':'Отлично! Назови, пожалуйста, свое имя...',
'surname':'Я в экстазе! Теперь напиши, пожалуйста, свою фамилию...',
'edu':'Раз уж мы так разоткровенничили, то где ты учился?',
'work':'Молодец! А где работаешь?',
'position':'Какая у тебя должность?'}

In [11]:
list(quetions_n.keys())[quetions_n[shelve.open(config.action_shelve)[str(354460030)]]]

'work'

In [10]:
list(quetions.keys())[-1]

'position'

In [12]:
shelve.open(config.action_shelve)[str(354460030)] in quetions_n.keys()

True